In [98]:
import numpy as np
from scipy.sparse.csgraph import minimum_spanning_tree
from time import time

In [83]:
loaded = np.load('../instances/iridium_weights_matricies.npz')
weights = loaded['w']
loaded.close()
N_CITIES = np.shape(weights)[1]

In [84]:
def compute_minimum_one_tree(weights):
    # get the mst collision matrix without the 0 element. 
    N_CITIES = np.shape(weights)[1]
    one = 0 # index of the 1 vertex (here 0).
    t = 0
    w_t = weights[t, 1:, 1:].copy()
    mst = minimum_spanning_tree(w_t) # Compute the MST over the graph without vertex 1.
    mst_graph = np.array(mst.toarray().astype(float)) # get the collision matrix
    np.shape(mst_graph)
    one_tree = np.insert(mst_graph, 0, np.zeros(N_CITIES - 1), axis=0) # insert the first row.
    one_tree = np.insert(one_tree, 0, np.zeros(N_CITIES), axis=1) # insert the first column.
    
    # respect the simmetry of the collision matrix. 
    for i in range(1, N_CITIES):
        for j in range(i + 1, N_CITIES):
            if one_tree[i,j] != 0:
                one_tree[j,i] = one_tree[i,j]
    # Find the two closest cities to vertex 0, belonging to T.
    closest_cities = np.argpartition(weights[t, 0, 1:], 2)
    closest_city1 = closest_cities[0]
    closest_city2 = closest_cities[1]
    # add the two links.
    one_tree[0, closest_city1] = weights[t, 0, closest_city1]
    one_tree[closest_city1, 0] = one_tree[0, closest_city1]
    one_tree[0, closest_city2] = weights[t, 0, closest_city2]
    one_tree[closest_city2, 0] = one_tree[0, closest_city2]
    return one_tree

In [85]:
def max_in_loop(c, one_tree_cm):
    """Find the loop, isolate it and return the value of the indices of the maximum weight inside the loop."""
    N_CITIES = np.shape(one_tree_cm)[0] # get the number of cities.
    collision_matrix = one_tree_cm.copy() # use a copy of the tree due to later modifications.
    
    #Create the list of all verticies.
    v = list(range(N_CITIES))
    # create the list of degree value for each vertex.
    degree_list = np.count_nonzero(collision_matrix, axis=1).tolist()
    
    try:
        while True: # Search for the cycle in collision_matrix. 
            c_i = degree_list.index(1) # get the index of degree == 1
            c_j = np.nonzero(collision_matrix[c_i])[0][0] # get its j value.
            
            # degrease the degree for the removed edges.
            degree_list[c_i] = 0
            degree_list[c_j] -= 1
            
            # update the collision matrix removing the edge.
            collision_matrix[c_i, c_j] = collision_matrix[c_j, c_i] = 0 
    except ValueError:
        pass
    
    # get the cycle indicies
    cycle = np.nonzero(degree_list)[0]
    
    if c in cycle: # test if the node belongs to the cycle.
        return np.unravel_index(np.argmax(collision_matrix), collision_matrix.shape)
    else: # else return None.
        return None

In [86]:
def compute_alpha_len(c1, c2, one_tree, weights):
    """Compute the value of the alpha length for the arc (c1, c2). """
    ot = one_tree.copy() # save a copy of one_tree.
    t = 0 # start from t = 0
    alpha_len = float('Inf') # initialize the alpha_len to infinite
    
    # case 1
    if one_tree[c1, c2] != 0:
        return 0
    elif c1 == 0 or c2 == 0:
        # take the largest of the two arcs incidents to node 0
        x = np.argmax(one_tree[0])
        # replace it with (c1, c2).
        one_tree[0, x] = one_tree[x, 0] = weights[t, c1, c2]
    else:
        one_tree[c1, c2] = one_tree[c2, c1] = weights[t, c1, c2] # add the link (c1,c2).
        max_arc_idx = max_in_loop(c1, one_tree)
        one_tree[idx[0], idx[1]] = one_tree[idx[1], idx[0]] = 0 # remove the heavier link in the loop.
    return np.sum(one_tree) / 2.0
        

In [87]:
def alpha_nearness(c1, c2, weights):
    # MAIN (it will be the alpha_nearness)
    # def alpha_nearness(c1, c2, weights)
    # compute the one minimum tree for the graph.
    one_tree = compute_minimum_one_tree(weights)
    l = np.sum(one_tree) / 2.0

    # compute the alpha distance:
    # (c1, c2) is the new arc.
    c1 = 1
    c2 = 2
    alpha_len = compute_alpha_len(c1, c2, one_tree, weights)

    # compute the alpha_nearness:
    return abs(alpha_len - l)

In [88]:
coll = np.zeros((5,5))
coll[0, 1] = 6.2
coll[1, 0] = 6.2
coll[0, 2] = 3.1
coll[2, 0] = 3.1
coll[1, 2] = 4.5
coll[2, 1] = 4.5
coll[3, 0] = 4.7
coll[0, 3] = 4.7
coll[3, 4] = 8.9
coll[4, 3] = 8.9

ret_val = max_in_loop(2, coll)
print("ret_val={}".format(ret_val))

ret_val=(0, 1)


In [89]:
coll = np.zeros((6,6))
coll[0, 2] = 9.8
coll[2, 0] = 9.8
coll[1, 2] = 3.1
coll[2, 1] = 3.1
coll[1, 3] = 4.8
coll[3, 1] = 4.8
coll[1, 5] = 7.1
coll[5, 1] = 7.1
coll[3, 5] = 6.3
coll[5, 3] = 6.3
coll[3, 4] = 5.4
coll[4, 3] = 5.4

res = [None, (1,5), None, (1,5), None, (1,5)]
for i in range(6):
    ret_val = max_in_loop(i, coll)
    if ret_val == res[i]:
        print("i={} : valid".format(i))
    else:
        print("i={} : INvalid".format(i))

i=0 : valid
i=1 : valid
i=2 : valid
i=3 : valid
i=4 : valid
i=5 : valid


In [95]:
arr = np.array([-1,5,7,-8,-1,5,7,-8,-1,5,7,-8,-1,5,7,-8,-1,5,7,-8,-1,5,7,-8,-1,5,7,-8,-1,5,7,-8])
t1 = time()
abs(arr)
t2 = time()
print(t2 - t1)
t1 = time()
np.abs(arr)
t2 = time()
print(t2 - t1)

8.0108642578125e-05
6.222724914550781e-05


In [99]:
arr[1:]

array([ 5,  7, -8, -1,  5,  7, -8, -1,  5,  7, -8, -1,  5,  7, -8, -1,  5,
        7, -8, -1,  5,  7, -8, -1,  5,  7, -8, -1,  5,  7, -8])